In [14]:
# Optimized US Stock & Options Screener - Much Faster Version
# ================================================================
# Key optimizations:
# 1. Start with popular/liquid stocks first (S&P 500, NASDAQ 100, etc.)
# 2. Add early filtering to reduce API calls
# 3. Better progress tracking
# 4. Reduced option chain processing
# 5. Error handling and retries

import yfinance as yf
import pandas as pd
import numpy as np
from tradingview_ta import TA_Handler, Interval, Exchange
from tqdm import tqdm
from joblib import Parallel, delayed
from scipy.stats import norm
import datetime
import time
import logging
import matplotlib.pyplot as plt
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ------------------------------
# OPTIMIZED Configuration
# ------------------------------
CONFIG = {
    'MAX_STOCK_PRICE_TO_OWN_100': 100000,
    'MAX_PREMIUM_PER_CONTRACT': 5000,
    'EARNINGS_WITHIN_DAYS': 14,
    'REQUIRE_TV_BUY': False,
    'EXPIRY_DAYS_MIN': 7,
    'EXPIRY_DAYS_MAX': 60,
    'MAX_WORKERS': 8,  # Reduced from auto-scale
    'TOP_N': 50,
    'OUTPUT_CSV_PATH': 'us_stock_options_screened.csv',
    'SCORE_WEIGHTS': {'tv':0.3, 'upside':0.3, 'delta':0.2, 'iv':0.2},
    'MIN_OPTION_VOLUME': 100,
    'MIN_OPTION_OPEN_INTEREST': 500,
    'QUICK_TEST_MODE': True,  # Start with popular stocks only
    'MAX_TICKERS_TO_PROCESS': 500  # Limit for faster testing
}

def timed(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        logging.info(f"Execution time for {func.__name__}: {end - start:.2f} seconds")
        return result
    return wrapper

# ------------------------------
# Get Popular/Liquid Stocks First (MUCH FASTER)
# ------------------------------
def get_popular_stocks():
    """Get popular stocks that are more likely to have good options activity"""
    popular_tickers = [
        # Tech giants
        'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'NFLX',
        # Financial
        'JPM', 'BAC', 'WFC', 'GS', 'MS', 'C',
        # Healthcare
        'JNJ', 'PFE', 'UNH', 'MRK', 'ABBV',
        # Others
        'SPY', 'QQQ', 'IWM', 'DIA', 'XLF', 'XLE', 'XLK', 'XLV',
        'AMD', 'INTC', 'CRM', 'ADBE', 'ORCL', 'IBM',
        'KO', 'PEP', 'WMT', 'HD', 'DIS', 'NKE'
    ]
    
    # Add S&P 500 tickers (subset)
    try:
        sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
        sp500_df = pd.read_html(sp500_url)[0]
        sp500_tickers = sp500_df['Symbol'].str.replace('.', '-').tolist()[:200]  # First 200
        popular_tickers.extend(sp500_tickers)
    except:
        logging.warning("Could not fetch S&P 500 list, using predefined popular stocks")
    
    return list(set(popular_tickers))[:CONFIG['MAX_TICKERS_TO_PROCESS']]

# ------------------------------
# Optimized TradingView rating with timeout
# ------------------------------
def get_tv_rating(symbol, timeout=10):
    try:
        handler = TA_Handler(
            symbol=symbol,
            screener='america',
            exchange='NASDAQ',
            interval=Interval.INTERVAL_1_DAY
        )
        analysis = handler.get_analysis()
        rating = analysis.summary['RECOMMEND']
        score_map = {
            'STRONG_BUY': 1.0,
            'BUY': 0.8,
            'NEUTRAL': 0.5,
            'SELL': 0.2,
            'STRONG_SELL': 0.0
        }
        return {
            'rating': rating,
            'score': score_map.get(rating, 0.5)
        }
    except Exception as e:
        logging.debug(f"TV rating failed for {symbol}: {str(e)}")
        return {'rating': None, 'score': 0.5}

# ------------------------------
# Optimized Yahoo Finance data fetching
# ------------------------------
def get_yf_data(symbol):
    try:
        stock = yf.Ticker(symbol)
        
        # Get basic info first for early filtering
        info = stock.info
        current_price = info.get('regularMarketPrice') or info.get('currentPrice')
        if not current_price or current_price * 100 > CONFIG['MAX_STOCK_PRICE_TO_OWN_100']:
            return None
        
        target_price = info.get('targetMeanPrice')
        earnings_date = None
        if 'earningsDate' in info and info['earningsDate']:
            earnings_date = info['earningsDate'][0] if isinstance(info['earningsDate'], list) else info['earningsDate']
        
        # Check earnings filter early
        if earnings_date:
            try:
                days_until_earnings = (pd.to_datetime(earnings_date) - datetime.datetime.now()).days
                if days_until_earnings > CONFIG['EARNINGS_WITHIN_DAYS']:
                    return None
            except:
                pass
        
        # Get option chains (limited to first 2 expiration dates for speed)
        options = stock.options
        if not options:
            return {
                'symbol': symbol,
                'price': current_price,
                'target_price': target_price,
                'earnings_date': earnings_date,
                'options': pd.DataFrame()
            }
        
        option_data = []
        for exp in options[:2]:  # Only first 2 expirations for speed
            try:
                calls = stock.option_chain(exp).calls
                if calls.empty:
                    continue
                    
                calls['expirationDate'] = exp
                calls['type'] = 'call'
                
                # Calculate time to expiry
                T = (pd.to_datetime(exp) - datetime.datetime.now()).days / 365.0
                if T <= 0:
                    continue
                
                S = current_price
                
                # Vectorized Greeks calculation for speed
                strikes = calls['strike'].values
                ivs = calls['impliedVolatility'].values
                
                # Replace zero or negative IVs
                ivs = np.where(ivs > 0, ivs, 0.3)
                
                d1 = (np.log(S / strikes) + 0.5 * ivs**2 * T) / (ivs * np.sqrt(T))
                
                calls['delta'] = norm.cdf(d1)
                calls['gamma'] = norm.pdf(d1) / (S * ivs * np.sqrt(T))
                calls['theta'] = -(S * norm.pdf(d1) * ivs) / (2 * np.sqrt(T))
                calls['vega'] = S * norm.pdf(d1) * np.sqrt(T)
                
                option_data.append(calls)
            except Exception as e:
                logging.debug(f"Option chain error for {symbol} exp {exp}: {str(e)}")
                continue
        
        options_df = pd.concat(option_data, ignore_index=True) if option_data else pd.DataFrame()
        
        return {
            'symbol': symbol,
            'price': current_price,
            'target_price': target_price,
            'earnings_date': earnings_date,
            'options': options_df
        }
        
    except Exception as e:
        logging.debug(f"YF data error for {symbol}: {str(e)}")
        return None

# ------------------------------
# Optimized screening function
# ------------------------------
def screen_ticker(symbol):
    try:
        # Get TradingView rating first for early filtering
        tv = get_tv_rating(symbol)
        if CONFIG['REQUIRE_TV_BUY'] and tv['rating'] not in ['BUY', 'STRONG_BUY']:
            return None
        
        # Get Yahoo Finance data
        yfdata = get_yf_data(symbol)
        if yfdata is None:
            return None
        
        # Process options if available
        if not yfdata['options'].empty:
            # Filter for liquid options
            liquid_options = yfdata['options'][
                (yfdata['options']['volume'] >= CONFIG['MIN_OPTION_VOLUME']) &
                (yfdata['options']['openInterest'] >= CONFIG['MIN_OPTION_OPEN_INTEREST'])
            ]
            
            if liquid_options.empty:
                return None
            
            # Get best option (highest IV for now)
            top_call = liquid_options.loc[liquid_options['impliedVolatility'].idxmax()]
            cop = top_call['delta'] * 100
            iv_score = min(top_call['impliedVolatility'] / 0.5, 1.0)  # Normalize IV
            delta_score = top_call['delta']
        else:
            cop = 50
            iv_score = 0.3
            delta_score = 0.5
            top_call = {}
        
        # Calculate upside potential
        upside = (yfdata['target_price'] / yfdata['price']) if yfdata['target_price'] else 1.0
        upside = min(upside, 2.0)  # Cap at 200% upside
        
        # Calculate combined score
        score = (CONFIG['SCORE_WEIGHTS']['tv'] * tv['score'] +
                CONFIG['SCORE_WEIGHTS']['upside'] * (upside - 1) +  # Normalize upside
                CONFIG['SCORE_WEIGHTS']['delta'] * delta_score +
                CONFIG['SCORE_WEIGHTS']['iv'] * iv_score)
        
        return {
            'symbol': symbol,
            'price': yfdata['price'],
            'target_price': yfdata['target_price'],
            'earnings_date': yfdata['earnings_date'],
            'tv_rating': tv['rating'],
            'score': score,
            'chance_of_profit': cop,
            'iv': top_call.get('impliedVolatility', 0),
            'delta': top_call.get('delta', 0),
            'strike': top_call.get('strike', 0),
            'premium': top_call.get('lastPrice', 0)
        }
        
    except Exception as e:
        logging.debug(f"Screen ticker error for {symbol}: {str(e)}")
        return None

# ------------------------------
# Optimized main screener function
# ------------------------------
@timed
def run_screener():
    """Run the optimized screener"""
    print("Starting optimized stock screener...")
    
    # Get ticker list
    if CONFIG['QUICK_TEST_MODE']:
        tickers = get_popular_stocks()
        print(f"Quick test mode: Processing {len(tickers)} popular stocks")
    else:
        tickers = fetch_us_stock_list()  # Your original function
        tickers = tickers[:CONFIG['MAX_TICKERS_TO_PROCESS']]
        print(f"Full mode: Processing {len(tickers)} stocks")
    
    print(f"Using {CONFIG['MAX_WORKERS']} workers")
    
    # Process tickers with progress bar
    results = []
    with ThreadPoolExecutor(max_workers=CONFIG['MAX_WORKERS']) as executor:
        # Submit all tasks
        future_to_ticker = {executor.submit(screen_ticker, ticker): ticker for ticker in tickers}
        
        # Process completed tasks with progress bar
        for future in tqdm(as_completed(future_to_ticker), total=len(tickers), desc="Screening stocks"):
            result = future.result()
            if result is not None:
                results.append(result)
    
    print(f"Found {len(results)} qualifying stocks")
    
    if not results:
        print("No stocks passed the screening criteria")
        return pd.DataFrame()
    
    # Convert to DataFrame and sort
    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values(by='score', ascending=False).head(CONFIG['TOP_N'])
    
    # Save results
    df_results.to_csv(CONFIG['OUTPUT_CSV_PATH'], index=False)
    print(f"Results saved to {CONFIG['OUTPUT_CSV_PATH']}")
    
    # Display top 10
    print(f"\nTop 10 Results:")
    print(df_results[['symbol', 'price', 'tv_rating', 'score', 'chance_of_profit']].head(10))
    
    # Create plot
    if len(df_results) > 0:
        plt.figure(figsize=(12, 6))
        plt.scatter(df_results['score'], df_results['chance_of_profit'], c='blue', alpha=0.7)
        
        # Annotate top 10 stocks
        top_10 = df_results.head(10)
        for _, row in top_10.iterrows():
            plt.annotate(row['symbol'], 
                        (row['score'], row['chance_of_profit']),
                        xytext=(5, 5), textcoords='offset points',
                        fontsize=8, alpha=0.8)
        
        plt.xlabel('Screener Score')
        plt.ylabel('Chance of Profit (%)')
        plt.title(f'Top {len(df_results)} Stocks: Score vs. Chance of Profit')
        plt.grid(True, alpha=0.3)
        plt.show()
    
    return df_results

# ------------------------------
# Debug function to see why stocks are being filtered out
# ------------------------------
def debug_ticker(symbol):
    """Debug why a ticker is being filtered out"""
    print(f"\n=== Debugging {symbol} ===")
    
    try:
        # Step 1: TradingView rating
        tv = get_tv_rating(symbol)
        print(f"1. TradingView rating: {tv['rating']} (score: {tv['score']})")
        if CONFIG['REQUIRE_TV_BUY'] and tv['rating'] not in ['BUY', 'STRONG_BUY']:
            print(f"   ❌ FILTERED OUT: TV rating not BUY/STRONG_BUY")
            return None
        
        # Step 2: Basic stock info
        stock = yf.Ticker(symbol)
        info = stock.info
        current_price = info.get('regularMarketPrice') or info.get('currentPrice')
        print(f"2. Current price: ${current_price}")
        
        if not current_price:
            print(f"   ❌ FILTERED OUT: No price data")
            return None
            
        if current_price * 100 > CONFIG['MAX_STOCK_PRICE_TO_OWN_100']:
            print(f"   ❌ FILTERED OUT: Price too high (${current_price * 100} > ${CONFIG['MAX_STOCK_PRICE_TO_OWN_100']})")
            return None
        
        # Step 3: Earnings check
        earnings_date = None
        if 'earningsDate' in info and info['earningsDate']:
            earnings_date = info['earningsDate'][0] if isinstance(info['earningsDate'], list) else info['earningsDate']
            print(f"3. Earnings date: {earnings_date}")
            
            if earnings_date:
                try:
                    days_until_earnings = (pd.to_datetime(earnings_date) - datetime.datetime.now()).days
                    print(f"   Days until earnings: {days_until_earnings}")
                    if days_until_earnings > CONFIG['EARNINGS_WITHIN_DAYS']:
                        print(f"   ❌ FILTERED OUT: Earnings too far ({days_until_earnings} > {CONFIG['EARNINGS_WITHIN_DAYS']} days)")
                        return None
                except Exception as e:
                    print(f"   ⚠️  Earnings date parsing error: {e}")
        else:
            print(f"3. No earnings date found")
        
        # Step 4: Options check
        options = stock.options
        print(f"4. Available option expirations: {len(options) if options else 0}")
        
        if not options:
            print(f"   ⚠️  No options available")
            return "NO_OPTIONS"
        
        # Check option liquidity
        option_data = []
        for exp in options[:2]:
            try:
                calls = stock.option_chain(exp).calls
                if not calls.empty:
                    print(f"   Expiration {exp}: {len(calls)} call options")
                    liquid_calls = calls[
                        (calls['volume'] >= CONFIG['MIN_OPTION_VOLUME']) &
                        (calls['openInterest'] >= CONFIG['MIN_OPTION_OPEN_INTEREST'])
                    ]
                    print(f"   Liquid calls (vol>={CONFIG['MIN_OPTION_VOLUME']}, OI>={CONFIG['MIN_OPTION_OPEN_INTEREST']}): {len(liquid_calls)}")
                    if not liquid_calls.empty:
                        option_data.append(liquid_calls)
                    
            except Exception as e:
                print(f"   Error processing {exp}: {e}")
        
        if not option_data:
            print(f"   ❌ FILTERED OUT: No liquid options found")
            return None
        
        print(f"   ✅ PASSED all filters!")
        return "PASSED"
        
    except Exception as e:
        print(f"   ❌ ERROR: {e}")
        return None

def debug_filters():
    """Debug the filtering process with popular stocks"""
    test_tickers = ['PGEN', 'CASI', 'PANW', 'AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NVDA', 'SPY', 'QQQ', 'AMD', 'META', 'AMZN']
    
    print("=== DEBUGGING FILTER CRITERIA ===")
    print(f"Current filter settings:")
    print(f"- REQUIRE_TV_BUY: {CONFIG['REQUIRE_TV_BUY']}")
    print(f"- MAX_STOCK_PRICE_TO_OWN_100: ${CONFIG['MAX_STOCK_PRICE_TO_OWN_100']}")
    print(f"- EARNINGS_WITHIN_DAYS: {CONFIG['EARNINGS_WITHIN_DAYS']}")
    print(f"- MIN_OPTION_VOLUME: {CONFIG['MIN_OPTION_VOLUME']}")
    print(f"- MIN_OPTION_OPEN_INTEREST: {CONFIG['MIN_OPTION_OPEN_INTEREST']}")
    
    results = {}
    for ticker in test_tickers:
        results[ticker] = debug_ticker(ticker)
        time.sleep(0.5)  # Be nice to APIs
    
    print(f"\n=== SUMMARY ===")
    passed = sum(1 for r in results.values() if r == "PASSED")
    no_options = sum(1 for r in results.values() if r == "NO_OPTIONS")
    filtered = sum(1 for r in results.values() if r is None)
    
    print(f"Passed all filters: {passed}")
    print(f"No options available: {no_options}")
    print(f"Filtered out: {filtered}")
    
    if passed == 0:
        print(f"\n🚨 RECOMMENDATION: Relax your filter criteria!")
        print(f"Try these settings:")
        print(f"- Set REQUIRE_TV_BUY = False")
        print(f"- Increase EARNINGS_WITHIN_DAYS to 30 or 45")
        print(f"- Reduce MIN_OPTION_VOLUME to 10")
        print(f"- Reduce MIN_OPTION_OPEN_INTEREST to 100")

# ------------------------------
# Relaxed configuration for testing
# ------------------------------
def use_relaxed_filters():
    """Apply more relaxed filter criteria"""
    global CONFIG
    CONFIG.update({
        'REQUIRE_TV_BUY': False,  # Don't require BUY rating
        'EARNINGS_WITHIN_DAYS': 45,  # More time until earnings
        'MIN_OPTION_VOLUME': 10,  # Lower volume requirement
        'MIN_OPTION_OPEN_INTEREST': 100,  # Lower OI requirement
        'MAX_STOCK_PRICE_TO_OWN_100': 2000  # Higher price limit
    })
    print("✅ Applied relaxed filter criteria")
    print("New settings:")
    for key in ['REQUIRE_TV_BUY', 'EARNINGS_WITHIN_DAYS', 'MIN_OPTION_VOLUME', 'MIN_OPTION_OPEN_INTEREST', 'MAX_STOCK_PRICE_TO_OWN_100']:
        print(f"  {key}: {CONFIG[key]}")

# ------------------------------
# Quick test function
# ------------------------------
def quick_test():
    """Test with just a few popular stocks"""
    test_tickers = ['PGEN', 'CASI', 'PANW', 'TSLA', 'NVDA']
    print("Quick test with 5 popular stocks...")
    
    results = []
    for ticker in tqdm(test_tickers, desc="Testing"):
        result = screen_ticker(ticker)
        if result:
            results.append(result)
        print(f"{ticker}: {'✓' if result else '✗'}")
    
    if results:
        df = pd.DataFrame(results)
        print("\nTest Results:")
        print(df[['symbol', 'price', 'tv_rating', 'score']])
    else:
        print("No results from test")

# First, let's debug why nothing is passing
print("🔍 Let's debug why no stocks are passing the filters...")
debug_filters()

🔍 Let's debug why no stocks are passing the filters...
=== DEBUGGING FILTER CRITERIA ===
Current filter settings:
- REQUIRE_TV_BUY: False
- MAX_STOCK_PRICE_TO_OWN_100: $100000
- EARNINGS_WITHIN_DAYS: 14
- MIN_OPTION_VOLUME: 100
- MIN_OPTION_OPEN_INTEREST: 500

=== Debugging PGEN ===
1. TradingView rating: None (score: 0.5)
2. Current price: $2.94
3. No earnings date found
4. Available option expirations: 3
   Expiration 2025-09-19: 5 call options
   Liquid calls (vol>=100, OI>=500): 3
   Expiration 2025-10-17: 7 call options
   Liquid calls (vol>=100, OI>=500): 5
   ✅ PASSED all filters!

=== Debugging CASI ===
1. TradingView rating: None (score: 0.5)
2. Current price: $2.38
3. No earnings date found
4. Available option expirations: 3
   Expiration 2025-09-19: 3 call options
   Liquid calls (vol>=100, OI>=500): 0
   Expiration 2025-10-17: 2 call options
   Liquid calls (vol>=100, OI>=500): 0
   ❌ FILTERED OUT: No liquid options found

=== Debugging PANW ===
1. TradingView rating: None 

In [9]:
run_screener()

Starting optimized stock screener...
Quick test mode: Processing 230 popular stocks
Using 24 workers


Screening stocks: 100%|██████████████████████████████████████████████████████████████| 230/230 [00:03<00:00, 75.44it/s]
2025-08-17 23:52:18,125 - INFO - Execution time for run_screener: 4.27 seconds


Found 0 qualifying stocks
No stocks passed the screening criteria


""


2025-08-17 23:19:45,312 - INFO - Execution time for test_function: 2.00 seconds


Execution time for test_function: 2.00 seconds
